In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


Looking in indexes: https://download.pytorch.org/whl/cu121


In [ ]:
!pip install transformers peft accelerate bitsandbytes datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.9 MB/s eta 0:00:00


In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM
tokenizer = LlamaTokenizer.from_pretrained('klyang/MentaLLaMA-chat-7B')

model = LlamaForCausalLM.from_pretrained('klyang/MentaLLaMA-chat-7B', device_map='auto',load_in_8bit=True )


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


In [ ]:
prompt = 'Consider this post: "work, it has been a stressful week! hope it gets better." Question: What is the stress cause of this post?'
inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
# Generate
generate_ids = model.generate(inputs.input_ids, max_length=512)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


'Consider this post: "work, it has been a stressful week! hope it gets better." Question: What is the stress cause of this post? Reasoning: The post mentions work as the cause of stress. nobody else mentions work in the post, and the poster mentions that it has been a stressful week. This indicates that the stress is directly related to their work. Therefore, the appropriate label for this post is Work.'

Evaluation

In [ ]:
!git clone https://github.com/SteveKGYang/MentalLLaMA.git
%cd MentalLLaMA/src
!python IMHI.py --model_path klyang/MentaLLaMA-chat-7B --batch_size 8 --model_output_path ../model_output --test_dataset IMHI --llama --cuda #batch_size 根据GPU调整
# evaluate baseline model

Cloning into 'MentalLLaMA'...
remote: Enumerating objects: 349, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 349 (delta 58), reused 95 (delta 36), pack-reused 224 (from 1)
Receiving objects: 100% (349/349), 13.50 MiB | 14.16 MiB/s, done.
Resolving deltas: 100% (153/153), done.


In [ ]:
import pandas as pd

# Load CSV using pandas
df = pd.read_csv("./DR_generated.csv")
df['answer'] = df.response.apply(lambda x: x.split("Reasoning:")[0].strip())
df['reason'] = df.response.apply(lambda x: x.split("Reasoning:")[1].strip())

In [ ]:
df.head()

,post,question,response,answer,reason
0,Post: I feel like I'm stuck in a loop that I c...,Question: The answer to the question \Does the...,yes. Reasoning: The post exhibits several emot...,yes.,The post exhibits several emotional indicators...
1,"Post: I don't know why, but I feel so empty al...",Question: The answer to the question \Does the...,yes. Reasoning: The post conveys a deep sense ...,yes.,The post conveys a deep sense of emotional num...
2,Post: I keep having these thoughts that I can'...,Question: The answer to the question \Does the...,yes. Reasoning: The post exhibits strong emoti...,yes.,The post exhibits strong emotional indicators ...
3,Post: I feel like I'm watching my life pass me...,Question: The answer to the question \Does the...,yes. Reasoning: The post conveys a deep sense ...,yes.,The post conveys a deep sense of emotional num...
4,"Post: I feel so alone, even when I'm surrounde...",Question: The answer to the question \Does the...,yes. Reasoning: The post exhibits strong emoti...,yes.,The post exhibits strong emotional indicators ...
